# Advanced RNA-seq Exploration

This notebook covers advanced analysis scenarios:
1. Multi-comparison workflows
2. Custom gene set analysis
3. Publication-quality figure customization
4. Advanced filtering and subsetting

## Setup

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import plotly.graph_objects as go

from rnaseq_parser import RNASeqParser
from de_analysis import DEAnalysisEngine
from visualizations import create_volcano_plot

## 1. Multi-Comparison Analysis

Analyze multiple treatment groups simultaneously.

In [ ]:
# Load data
parser = RNASeqParser()
result = parser.parse("../tests/data/sample_counts.csv")

# Define multiple conditions
sample_conditions = {
    "Sample_1": "Control",
    "Sample_2": "Control",
    "Sample_3": "Treatment_A",
    "Sample_4": "Treatment_A",
    "Sample_5": "Treatment_B",
    "Sample_6": "Treatment_B",
    "Sample_7": "Treatment_C",
    "Sample_8": "Treatment_C",
    "Sample_9": "Treatment_C",
    "Sample_10": "Treatment_C"
}

metadata_df = pd.DataFrame({'condition': sample_conditions})

# Define all pairwise comparisons
comparisons = [
    ("Treatment_A", "Control"),
    ("Treatment_B", "Control"),
    ("Treatment_C", "Control"),
    ("Treatment_B", "Treatment_A"),
    ("Treatment_C", "Treatment_A")
]

# Run all comparisons
engine = DEAnalysisEngine()
de_results = engine.run_all_comparisons(
    result.expression_df,
    metadata_df,
    comparisons=comparisons,
    design_factor="condition"
)

# Summary table
summary = []
for (test, ref), de_result in de_results.items():
    sig_count = (de_result.results_df['padj'] < 0.05).sum()
    up_count = ((de_result.results_df['padj'] < 0.05) & 
                (de_result.results_df['log2FoldChange'] > 1)).sum()
    down_count = ((de_result.results_df['padj'] < 0.05) & 
                  (de_result.results_df['log2FoldChange'] < -1)).sum()
    
    summary.append({
        'Comparison': f"{test} vs {ref}",
        'Significant': sig_count,
        'Upregulated': up_count,
        'Downregulated': down_count
    })

pd.DataFrame(summary)

### Venn Diagram of Overlapping Genes

Find genes significant across multiple comparisons.

In [ ]:
# Get significant genes for each comparison
sig_genes = {}
for (test, ref), de_result in de_results.items():
    sig = de_result.results_df[de_result.results_df['padj'] < 0.05]
    sig_genes[f"{test}_vs_{ref}"] = set(sig['gene'].tolist())

# Find overlaps
print("Overlap analysis:")
comp_names = list(sig_genes.keys())
for i in range(len(comp_names)):
    for j in range(i+1, len(comp_names)):
        overlap = sig_genes[comp_names[i]] & sig_genes[comp_names[j]]
        print(f"{comp_names[i]} ∩ {comp_names[j]}: {len(overlap)} genes")

# Genes significant in ALL comparisons
all_overlap = set.intersection(*sig_genes.values())
print(f"\nGenes significant in ALL comparisons: {len(all_overlap)}")
if all_overlap:
    print(list(all_overlap)[:10])  # Show first 10

## 2. Custom Gene Set Analysis

Analyze expression of user-defined gene lists.

In [ ]:
# Define custom gene set
my_genes = [
    "COL1A1", "COL3A1", "ELN",  # ECM genes
    "IL6", "IL1B", "TNFA",       # Inflammatory genes
    "TYR", "TYRP1", "DCT"        # Melanogenesis genes
]

# Get expression data for these genes
de_result = de_results[("Treatment_A", "Control")]
gene_expr = de_result.log_normalized_counts[my_genes]

# Calculate z-scores across samples
z_scores = (gene_expr - gene_expr.mean()) / gene_expr.std()

# Mean z-score per sample
sample_scores = z_scores.mean(axis=1)

# Mean per condition
condition_scores = {}
for condition in set(sample_conditions.values()):
    samples = [s for s, c in sample_conditions.items() if c == condition]
    condition_scores[condition] = sample_scores.loc[samples].mean()

print("Custom gene set scores (z-score normalized):")
for condition, score in sorted(condition_scores.items()):
    print(f"  {condition}: {score:.2f}")

## 3. Publication Figure Customization

Advanced Plotly customization for publication-quality figures.

In [ ]:
# Create base volcano plot
de_result = de_results[("Treatment_A", "Control")]
fig = create_volcano_plot(de_result.results_df)

# Customize for publication
fig.update_layout(
    title={
        'text': "Treatment A vs Control",
        'font': {'size': 20, 'family': 'Arial', 'color': 'black'}
    },
    xaxis={
        'title': {'text': 'log<sub>2</sub> Fold Change', 'font': {'size': 16}},
        'showgrid': True,
        'gridcolor': 'lightgray',
        'zeroline': True,
        'zerolinecolor': 'black',
        'zerolinewidth': 1
    },
    yaxis={
        'title': {'text': '-log<sub>10</sub>(adjusted p-value)', 'font': {'size': 16}},
        'showgrid': True,
        'gridcolor': 'lightgray'
    },
    plot_bgcolor='white',
    width=800,
    height=600,
    font={'family': 'Arial', 'size': 12, 'color': 'black'}
)

# Add gene labels for top hits
top_genes = de_result.results_df.nsmallest(10, 'padj')
for _, row in top_genes.iterrows():
    fig.add_annotation(
        x=row['log2FoldChange'],
        y=-np.log10(row['padj']),
        text=row['gene'],
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=1,
        arrowcolor='black',
        ax=20,
        ay=-20,
        font={'size': 10, 'color': 'black'}
    )

fig.show()

## 4. Advanced Filtering

Apply stringent criteria to identify high-confidence hits.

In [ ]:
# Stringent filtering
de_result = de_results[("Treatment_A", "Control")]

stringent_hits = de_result.results_df[
    (de_result.results_df['padj'] < 0.01) &  # More stringent p-value
    (de_result.results_df['log2FoldChange'].abs() > 2) &  # Larger fold change
    (de_result.results_df['baseMean'] > 100)  # Exclude low-expression genes
]

print(f"Stringent hits: {len(stringent_hits)} genes")
print(f"\nTop 20 by fold change:")
stringent_hits.nlargest(20, 'log2FoldChange')[['gene', 'log2FoldChange', 'padj', 'baseMean']]

### Subset by Biological Category

Filter for specific gene families or pathways.

In [ ]:
# Example: Find all collagen genes
collagen_genes = de_result.results_df[
    de_result.results_df['gene'].str.startswith('COL')
]

print(f"Collagen genes detected: {len(collagen_genes)}")
collagen_genes.sort_values('padj')[['gene', 'log2FoldChange', 'padj']].head(10)

### Export Filtered Results

In [ ]:
# Export stringent hits to CSV
stringent_hits.to_csv("stringent_hits.csv", index=False)
print("✓ Exported stringent_hits.csv")

# Export collagen genes
collagen_genes.to_csv("collagen_genes.csv", index=False)
print("✓ Exported collagen_genes.csv")

## Summary

This notebook demonstrated:
1. ✓ Multi-comparison workflows with overlap analysis
2. ✓ Custom gene set scoring with z-score normalization
3. ✓ Publication-quality figure customization
4. ✓ Advanced filtering strategies

**Tips for Your Analysis:**
- Adjust thresholds based on your experimental design
- Use overlap analysis to find robust biomarkers
- Customize figures to match journal requirements
- Apply biological knowledge to filter results